# Quantum Dynamics 1D

## General imports and initialization

In [1]:
import numpy as np
import scipy as sp
from scipy import sparse
from scipy.sparse import linalg
import matplotlib.pyplot as plt
import timeit
from matplotlib import animation
#matplotlib inline
%matplotlib auto

#A framework to create mp4 files, must be installed on this location on the computer
plt.rcParams['animation.ffmpeg_path'] = "C:/FFMPEG/bin/ffmpeg.exe"

Using matplotlib backend: Qt4Agg


In [2]:
#Number of points in x-direction 
Nx = 2001

#Create x-grid
xrange = 40
x = np.linspace(-xrange,xrange,Nx)
dx = xrange*2/(Nx-1)

#Make corresponding k-grid
k_four = np.fft.fftfreq(Nx, dx)*2*np.pi

In [49]:
#Create potentiallandscape
def potential_barrier(x, width, height, center_x):
    V_vec = height*(0.5*(np.sign((x-center_x)+width/2)) - 
                    0.5*(np.sign((x-center_x)-width/2)))    
    return V_vec

#Create Hamiltonian, needed for Crank-Nicolson
def HamiltonianOperator(V_vec,Nx,dx):
    H = dx**-2 * (2 * sparse.eye(Nx,Nx,0)-sparse.eye(Nx,Nx,1) - 
                  sparse.eye(Nx,Nx,-1)) + sparse.diags(V_vec,0) + 1j*0
    H[0,Nx-1]=-1*dx**-2
    H[Nx-1,0]=-1*dx**-2
    return H

### For Crank-Nicolson

In [54]:
#Create time-grid
Nt = 4000
dt = 0.001
t = np.linspace(0,Nt*dt,Nt)

### For Trotterization

In [59]:
#Create time-grid
dt = 0.0001
Nt = 70000
t = np.linspace(0,Nt*dt,Nt)

### Initialization of Wave packet

In [60]:
psi_x = np.zeros((Nx, Nt), dtype = complex)
psi_k = np.zeros((Nx, Nt), dtype = complex)

#Begin with a normalized wavepacket in momentum space, width_k = 1/width, centered
#around k0 in momentum space, x0 in real space.
width = 2.5
k0 = 7.5
x0 = 0
psi_k[:,0] = ((width / np.pi)**(1/4)
            * np.exp(-0.5 * (width * (k_four - k0)) ** 2 - 1j * (k_four - k0) * x0))

#Transform to have a wavepacket in x-space
psi_x[:,0] = np.fft.fftshift(np.fft.ifft(psi_k[:,0], norm = "ortho"))

#We make a potential barrier which is just 4/3 * average energy of the wavepacket
#height = 6/5 * p0^2/2m = 4/3 * k0^2 in natural units
height = 5/4 * k0**2
V_pot = potential_barrier(x, width = 0.5, height = height, center_x = 10)

## Simulation 

### Run block for Crank Nicolson 

In [56]:
HD = HamiltonianOperator(V_pot,Nx,dx)

for i in range(Nt-1):
    psi_x[:,i+1]=(sparse.eye(Nx,Nx,0)-1j*dt*HD/2)*linalg.spsolve(sparse.eye(Nx,Nx,0)+
                                                                 1j*dt*HD/2,psi_x[:,i])
    
    #Get psi_k, by Fourier transforming wavepacket to momentum space
    psi_k[:,i+1] = np.fft.fft(psi_x[:,i+1], norm = "ortho")

C:\Users\TU Delf SID\Anaconda3\lib\site-packages\scipy\sparse\compressed.py:739: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


### Run block for Trotterization

In [61]:
#Diagonal trotterization operator in momentum space
exponentP = np.exp(-1j*dt*abs(k_four)**2)

#Diagonal trotterization operator in real space

exponentV = np.exp(-1j*dt*V_pot)

#For each timestep, perform trotterization step
for i in range(Nt-1):
    
    #The first Fourier transformation to momentum space
    term1 = np.fft.fft(exponentV*psi_x[:,i], norm = "ortho")
    
    #Inverse Fourier transform, back to real space
    psi_x[:,i+1] = np.fft.ifft(exponentP*term1, norm = "ortho")
    
    #Get psi_k, by Fourier transforming wavepacket to momentum space
    psi_k[:,i+1] = np.fft.fft(psi_x[:,i+1], norm = "ortho")
   
psi_x *= np.sqrt(max(k_four)/max(x))

## Animation

In [63]:
#We create the figures and set limits on the axis
#Figure 1: x-space
#Figure 2: k-space
fig, (ax1, ax2) = plt.subplots(2,1)
ax1.set_xlim(x[0], x[-1])
ax1.set_ylim(0, 1.2*(abs(psi_x[:,0]**2)).max())
ax2.set_xlim(-2 * k0, 2*k0)
ax2.set_ylim(0, abs(psi_k[:,0]**2).max())

#Add potential barrier in k-space
plt.axvline(x=np.sqrt(max(V_pot)), ymin=0, ymax = 1, linewidth=2, color='k')

#Set labels and titles
ax1.set_xlabel('x')
ax1.set_ylabel(r'$|\psi(x)|^2$')
ax2.set_xlabel('k')
ax2.set_ylabel(r'$|\psi(k)|^2$')
ax1.set_title('Real space Wavepacket and potential', fontweight = 'bold')
ax2.set_title('Momentum Space Wavepacket and potential barrier', fontweight = 'bold')

#lines[0]: psi(x)
#lines[1] = Potential landscape
#lines[2]  = psi(k)
lines = [ax1.plot([], [])[0], ax1.plot([], [])[0], ax2.plot([], [], color = 'red')[0]]


#These functions will be repeatedly called to create an animation,
def animate(i):
    """Given an index i, return the wavefunction in real space and wavefunction and the 
    potential barrier in momentum space at the corresponding time"""
    lines[0].set_ydata(abs(psi_x[:,i])**2)
    lines[1].set_ydata(V_pot)
    lines[2].set_ydata(abs(psi_k[:,i])**2)
    return lines

# Init only required to set first frame.
def init():    
    """Return initial wavefunction in real space, wavefunction and potential barrier in 
    momentum space at time t = 0"""
    lines[0].set_data(x, abs(psi_x[:,0])**2)
    lines[1].set_data(x,V_pot)
    lines[2].set_data(k_four, abs(psi_k[:,0])**2)
    return lines

#The animation is run, different frames taken for different methodes to compent
anim = animation.FuncAnimation(fig, animate,  np.arange(1, Nt + 1, 10**(-2)/dt), 
                               init_func=init,interval=30, blit=True)

#Call the FFMPEG platform and save the animation as mp4 file, uncomment if you want to SAVE
FFwriter = animation.FFMpegWriter()
#anim.save(r'.\Animations\1D_Trotterization.mp4', writer 
           #= FFwriter, fps=None, extra_args=['-vcodec', 'libx264'])

#Show animation
#plt.show()

C:\Users\TU Delf SID\Anaconda3\lib\site-packages\ipykernel\__main__.py:31: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\TU Delf SID\Anaconda3\lib\site-packages\ipykernel\__main__.py:33: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\TU Delf SID\Anaconda3\lib\site-packages\ipykernel\__main__.py:31: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\TU Delf SID\Anaconda3\lib\site-packages\ipykernel\__main__.py:33: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
